In [2]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=303
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs'
TARGET_SPECIES='N'
####################################################

os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()




def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()





av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    eta0s.append(eta0)
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    
    
    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")


    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f1
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(os.getcwd()))
print("==================================")
print("""
target_atom_labels=\t{}\n
av_theta_xs=\t{}\n    
av_theta_ys=\t{}\n  
av_theta_y2s=\t{}\n
av_theta_x2s=\t{}\n 
cq_coef0s=\t{}\n
cq_coef1s=\t{}\n
eta_coefs=\t{}\n
cq0s=\t{}\n
cqprime0s=\t{}\n
cqprime1s=\t{}\n 
eta0s=\t{}\n
etaprimes=\t{}\n
fq0s=\t{}\n 
fqprimes=\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()


print("eta0s\n---------")
for item in target_atom_labels:
    print(item, eta0s[target_atom_labels.index(item)])
print()





endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()




Begin on Mon Feb 19 14:43:46 2018

Working dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs


TEMP:            303
TARGET_DIR:      /Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs
TARGET_SPECIES:  N


There are 901 efg files in this directory.

901 efg files opened without error.

The indices of N atoms are:
1 3 4 6 23 25 26 28 37 39 40 42 51 53 54 56 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
skipped:891
skipped:892
skipped:893
skipped:894
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	1
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs
<tx>:	 -0.00318607949883 	<ty>:	 

Fetching the angles in file efg.{}.out
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
skipped:891
skipped:892
skipped:893
skipped:894
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	26
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs
<tx>:	 0.0106887409121 	<ty>:	 -0.00160887254098 
 <tx^2>	: 0.116765244566 	<ty^2>: 0.0380409871694 

cq0=	-0.7064
eta0=	0.64886
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs
T=303K, nuclear site: 26  =
1-3/2(<tx^2>+<ty^2>)		=0.7677906523968041
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.8061014186376247

Cq0:		-0.7064
Cq' from coefficient 1-3/2(<>+<>)

skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
skipped:891
skipped:892
skipped:893
skipped:894
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	51
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs
<tx>:	 -0.012661760667 	<ty>:	 0.00545174321328 
 <tx^2>	: 0.091217661754 	<ty^2>: 0.0203360177159 

cq0=	-0.7601
eta0=	0.61402
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930/scfs
T=303K, nuclear site: 51  =
1-3/2(<tx^2>+<ty^2>)		=0.8326694807951057
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.8653115410993144

Cq0:		-0.7601
Cq' from coefficient 1-3/2(<>+<>):		-0.6329120723523599
Cq' from coeffici

In [1]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

target_atom_labels=	[1, 3, 4, 6, 23, 25, 26, 28, 37, 39, 40, 42, 51, 53, 54, 56]

av_theta_xs=	[-0.0031860794988272773, -0.023140966548382338, 0.019964964154457684, 0.0006421866819625383, -0.020325524223731118, 0.0061088229364159342, 0.010688740912080361, 0.002762956962464693, 0.018260967985238935, 0.023310039201913206, -0.0010448397968295632, -0.0044874264225630133, -0.012661760666950071, -0.0025922658252539193, -0.026497753128843549, 0.012927353822258538]
    
av_theta_ys=	[-0.0041328637274362021, -0.012772166715557694, -0.0051333464742900322, -0.012042810014313305, -0.0027513281139652304, -0.012882068726275901, -0.0016088725409773635, -0.0012858958928121654, 0.012497014911533329, -0.012872630563377495, 0.0046020667591151017, 0.000747959830256604, 0.0054517432132822619, -0.017427219705507719, -0.0044748154589689393, -0.0063657398369723296]
  
av_theta_y2s=	[0.0048905181585550796, 0.030062534486452647, 0.003158305398920612, 0.02357795397231852, 0.024402973454417123, 0.0037186728628475869, 0.038040987169378294, 0.0051237106004291258, 0.020797631869236711, 0.0050132553078318263, 0.036133276624415199, 0.0047395232448135729, 0.020336017715911481, 0.0053494210449344659, 0.033564204154515637, 0.0050566835366570454]

av_theta_x2s=	[0.0039924541323849497, 0.046219646317509112, 0.0063955179944490169, 0.093700612388637278, 0.075590801470272112, 0.0061854426492596532, 0.1167652445660856, 0.0026188691099136457, 0.085339941365998664, 0.0072279974213683455, 0.096427263039189873, 0.0065168655819231426, 0.09121766175401802, 0.0038563132462374468, 0.080774092649211357, 0.0066365216571907136]
 
cq_coef0s=	[0.98667554156358994, 0.88557672879405736, 0.98566926490994555, 0.82408215045856625, 0.85000933761296615, 0.98514382673183909, 0.76779065239680411, 0.98838613043448587, 0.84079364014714697, 0.98163812090619973, 0.80115919050459239, 0.98311541675989489, 0.83266948079510572, 0.98619139856324212, 0.82849255479440953, 0.98246019220922831]

cq_coef1s=	[0.98633319731165381, 0.89344653482472836, 0.98680285583058469, 0.85636311626051864, 0.87358210023168759, 0.98600759084024914, 0.80610141863762474, 0.98743127859619761, 0.87050569232398134, 0.98241366822580745, 0.83052718540750092, 0.9837929441021569, 0.86531154109931441, 0.98566857193645041, 0.85146701098091071, 0.98306243045037112]

eta_coefs=	[]

cq0s=	[-6.2245, -0.7063, -6.1177, -0.7601, -0.7601, -6.1179, -0.7064, -6.2246, -0.7601, -6.1177, -0.7063, -6.2245, -0.7601, -6.1179, -0.7064, -6.2246]

cqprime0s=	[-6.1415619084625659, -0.62548284354724271, -6.0300288619395737, -0.6263848425635562, -0.6460920975196156, -6.0270114175627176, -0.54236731685310247, -6.1523083075025005, -0.63908724587584642, -6.0053675322678579, -0.56585873625339367, -6.1194019116219653, -0.63291207235235991, -6.0334203572700584, -0.58524714070677086, -6.1154217124255625]

cqprime1s=	[-6.1394309866663894, -0.63104128754670563, -6.0369638311147682, -0.65092160466962024, -0.66400975438610577, -6.0322958400015603, -0.5694300421256181, -6.1463647367498915, -0.66167137673545817, -6.0101120981050222, -0.5866013510533179, -6.1236191805638756, -0.65772330238958887, -6.0302217562500093, -0.60147629655691537, -6.1191704045813795]
 
eta0s=	[0.50827, 0.64944, 0.4669, 0.6138, 0.61402, 0.46688, 0.64886, 0.50827, 0.6138, 0.4669, 0.64944, 0.50827, 0.61402, 0.46688, 0.64886, 0.50827]

etaprimes=	[]

fq0s=   [np.array([-1.56078584, -3.82577851, -5.38656435]),np.array([-0.20310679, -0.36755874, -0.57066553]),np.array([-1.40771024, -3.81866653, -5.22637677]),np.array([-0.19223751, -0.37366988, -0.56590739]),np.array([-0.19835673, -0.38539071, -0.58374744]),np.array([-1.40694555, -3.81678579, -5.22373134]),np.array([-0.17596023, -0.31879537, -0.4947556 ]),np.array([-1.56351687, -3.83247279, -5.39598967]),np.array([-0.19613588, -0.3812475 , -0.57738337]),np.array([-1.40195305, -3.80304912, -5.20500217]),np.array([-0.18374565, -0.33252123, -0.51626688]),np.array([-1.5551542 , -3.81197433, -5.36712854]),np.array([-0.19431034, -0.37752889, -0.57183922]),np.array([-1.40844165, -3.82084444, -5.22928609]),np.array([-0.18987173, -0.34399949, -0.53387122]),np.array([-1.5541427 , -3.80949494, -5.36363763])]

fqprimes=	[]

TEMP=303
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930'
os.chdir(TARGET_DIR)
os.getcwd()
x=md.Md('md.in','md.out')
vol=x.cell_volume
print(TARGET_DIR,"\n","-"*len(TARGET_DIR),"\n")

def get_inds(target, inds):
    result = []
    for ind in inds:
        result.append(target[ind])
    return result

axial_inds = [ 0,  7, 11, 15]
equit_inds = [ 2,  5,  9, 13]
ax         = axial_inds
eq         = equit_inds

axial_cq0s = get_inds(cq0s, ax)
axial_eta0s= get_inds(eta0s, ax)
axial_k0s  = get_inds(cq_coef0s, ax)
axial_k1s  = get_inds(cq_coef1s, ax)
axial_cq   = np.average(get_inds(cq0s, ax))
axial_eta  = np.average(get_inds(eta0s, ax))
axial_k0   = np.average(get_inds(cq_coef0s, ax))
axial_k1   = np.average(get_inds(cq_coef1s, ax))


equit_cq0s = get_inds(cq0s, eq)
equit_eta0s= get_inds(eta0s, eq)
equit_k0s  = get_inds(cq_coef0s, eq)
equit_k1s  = get_inds(cq_coef1s, eq)
equit_cq   = np.average(get_inds(cq0s, eq))
equit_eta  = np.average(get_inds(eta0s, eq))
equit_k0   = np.average(get_inds(cq_coef0s, eq))
equit_k1   = np.average(get_inds(cq_coef1s, eq))



print("cell volume:", vol,"\n")
print("AXIAL","\n","-"*len(TARGET_DIR))
print("avg Cq0:", np.average(axial_cq0s),"\t",axial_cq)
print("avg eta0:", np.average(axial_eta0s),"\t",axial_eta)
print("avg coef0:",np.average(axial_k0s),"\t",axial_k0)
print("avg coef1:",np.average(axial_k1s),"\t",axial_k1)
print("using coef0:",axial_k0*axial_cq)
print("using coef1:",axial_k1*axial_cq)


ffunc=nqr.f1
fraw=ffunc(axial_cq,axial_eta)
f0=ffunc(axial_k0*axial_cq,axial_eta)
f1=ffunc(axial_k1*axial_cq,axial_eta)
exptcq=6.029
expteta=0.5205
exptfq=[5.306,       3.737]
print("expt cq:\t", exptcq)
print("raw freq:\t",list(reversed(fraw))[:-1])

print("expt fq:\t", exptfq)
print("adjusted f0s:\t",list(reversed(f0))[:-1])
print("adjusted f1s:\t",list(reversed(f1))[:-1])
print("\n\n")

print("EQUITORIAL","\n","-"*len(TARGET_DIR))
print("avg Cq0:", np.average(equit_cq0s),"\t",equit_cq)
print("avg eta0:", np.average(equit_eta0s),"\t",equit_eta)
print("avg coef0:",np.average(equit_k0s),"\t",equit_k0)
print("avg coef1:",np.average(equit_k1s),"\t",equit_k1)
print("using coef0:",equit_k0*equit_cq)
print("using coef1:",equit_k1*equit_cq)


ffunc=nqr.f1
fraw=ffunc(equit_cq,equit_eta)
f0=ffunc(equit_k0*equit_cq,equit_eta)
f1=ffunc(equit_k1*equit_cq,equit_eta)
exptcq=5.795
expteta=0.4980
exptfq=[5.068,       3.625]
print("expt cq:\t", exptcq)
print("raw freq:\t",list(reversed(fraw))[:-1])

print("expt fq:\t", exptfq)
print("adjusted f0s:\t",list(reversed(f0))[:-1])
print("adjusted f1s:\t",list(reversed(f1))[:-1])


/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792930 
 -------------------------------------------------------- 

cell volume: 517.450150179 

AXIAL 
 --------------------------------------------------------
avg Cq0: -6.22455 	 -6.22455
avg eta0: 0.50827 	 0.50827
avg coef0: 0.985159320242 	 0.985159320242
avg coef1: 0.985154962615 	 0.985154962615
using coef0: -6.13217344681
using coef1: -6.13214632255
expt cq:	 6.029
raw freq:	 [-5.4593505071249995, -3.8774744928749993]
expt fq:	 [5.306, 3.737]
adjusted f0s:	 [-5.3783300345609897, -3.8199301356556519]
adjusted f1s:	 [-5.3783062447494281, -3.8199132390692543]



EQUITORIAL 
 --------------------------------------------------------
avg Cq0: -6.1178 	 -6.1178
avg eta0: 0.46689 	 0.46689
avg coef0: 0.984660652778 	 0.984660652778
avg coef1: 0.985223171708 	 0.985223171708
using coef0: -6.02395694156
using coef1: -6.02739831988
expt cq:	 5.795
raw freq:	 [-5.3024349104999997, -3.8742650895000006]
expt fq:	 [5.068, 3.625]
adjusted f0s: